<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# SPARK

_This task is very similar to the one you have already done on Hadoop. It should be so, appreciate how much easier it is to solve on Spark._

We will use the logs of listening to music artists in the Yandex.Music service.

The `events.csv` file contains entries like `User,Artist,Number of plays,Number of skips`:
```csv
userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1
```

We need to do the following:
1. **Leave in the data only those users for whom the sum of plays is strictly greater than 2000. How many such users?**
2. **In the data filtered at the first step, find the 5 most popular performers by the number of users (identifiers).**

Details:
1. Let's assume that a single user's playlist always fits in memory.

Save the solution to the `result.json` file. 

In [1]:
import json

In [1]:
# file content example
! head -n 5 yandex_music/events.csv

userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1


In [2]:
# copy files to HDFS
! hadoop fs -copyFromLocal yandex_music /
! hadoop fs -ls -h /yandex_music

Found 3 items
-rw-r--r--   1 jovyan supergroup        254 2023-12-21 17:43 /yandex_music/README.txt
-rw-r--r--   1 jovyan supergroup      3.7 M 2023-12-21 17:43 /yandex_music/artists.jsonl
-rw-r--r--   1 jovyan supergroup     47.6 M 2023-12-21 17:43 /yandex_music/events.csv


In [3]:
# https://spark.apache.org/docs/latest/rdd-programming-guide.html
# http://spark.apache.org/docs/latest/sql-getting-started.html

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-12-21 17:43:21,257 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# load csv as Spark DataFrame
events = se.read.csv("hdfs:///yandex_music/events.csv", header=True)  # в первой строке у нас заголовок
events.registerTempTable("events")
events.limit(5).toPandas()

,userId,artistId,plays,skips
0,0,335,1,0
1,0,708,1,0
2,0,710,2,1
3,0,815,1,1
4,0,880,1,1


In [5]:
# we can convert this DataFrame to RDD
events.rdd.take(5)

[Row(userId='0', artistId='335', plays='1', skips='0'),
 Row(userId='0', artistId='708', plays='1', skips='0'),
 Row(userId='0', artistId='710', plays='2', skips='1'),
 Row(userId='0', artistId='815', plays='1', skips='1'),
 Row(userId='0', artistId='880', plays='1', skips='1')]

In [11]:
# Filter users with a sum of plays greater than 2000
filtered_users = events.groupBy("userId").agg({"plays": "sum"}).filter("sum(plays) > 2000")
filtered_users.limit(100).toPandas()

,userId,sum(plays)
0,125,3176.0
1,451,2677.0
2,591,2959.0
3,613,2793.0
4,334,2714.0
...,...,...
95,404,2262.0
96,464,3687.0
97,23,2330.0
98,533,3441.0


In [12]:
# Get the artistIds for filtered users
filtered_artistIds = events.join(filtered_users, "userId", "inner").select("userId", "artistId")

In [13]:
# Count the number of unique users for each artist and get the top 5
top_artists = filtered_artistIds.groupBy("artistId").agg({"userId": "count"}).orderBy("count(userId)", ascending=False).limit(5)

In [14]:
# Collect the results and prepare for saving to JSON
result = {
    "q1": filtered_users.count(),
    "q2": top_artists.select("artistId").rdd.flatMap(lambda x: x).collect()
}

# Save the result to a JSON file
se.createDataFrame([Row(**result)]).write.json("hdfs:///yandex_music/result.json")

In [17]:
res = {'q1': 1705, 'q2': [11368, 3629, 259, 44148, 23524]}

In [20]:
result = json.dumps(res)
print(result)

{"q1": 1705, "q2": [11368, 3629, 259, 44148, 23524]}


In [21]:
f = open("result.json", "w")
f.write(result)
f.close()

In [22]:
!curl -F file=@result.json 51.250.48.170:80/MDS-LSML1/name/w3/1

1.0
Correct q1 answer! Correct q2 answer!


In [ ]:
# stop Spark (and YARN application)
sc.stop()